# Defs

In [27]:
from __future__ import division
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import pandas as pd
from collections import Counter
from nltk.tokenize import TweetTokenizer
tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
from sklearn.metrics import classification_report

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='VALID')

def max_pool(x):
    return tf.nn.max_pool(x, [1, 41, 1, 1], strides=[1,1,1,1], padding='VALID')


def add_filter(num_of_words):
    W_conv = weight_variable([num_of_words,embedding_size,1,10])
    b_conv = bias_variable([10])
    h_conv = tf.nn.relu(conv2d(embed_chars_expanded, W_conv) + b_conv)
    h_pool = max_pool(h_conv)
    
    return h_pool, h_conv, W_conv

# Create the Network

In [2]:
vocab = np.load('/home/dima/Data/sentiment_vocab.npy')
len(vocab)

21656

In [3]:
vocab_size = len(vocab)
embedding_size = 300
sequence_size = 42
num_classes = 3

print vocab_size, embedding_size, sequence_size, num_classes

21656 300 42 3


In [5]:
input_x = tf.placeholder(tf.int32, [None, sequence_size], name="input_x")
input_y = tf.placeholder(tf.float32, [None, num_classes], name="input_y")

W_embed = tf.Variable(tf.random_uniform([vocab_size, embedding_size], -1.0, 1.0), name="W_embed")
embed_chars = tf.nn.embedding_lookup(W_embed, input_x)
embed_chars_expanded = tf.expand_dims(embed_chars, -1)

filter = add_filter(2)

h_pool = tf.concat(3, [filter[0]])
h_pool_flat = tf.reshape(h_pool, [-1, 10])

fc_W = tf.Variable(tf.truncated_normal([10, 3], stddev=0.1), name="fc_W")
fc_b = tf.Variable(tf.constant(0.1, shape=[3]), name="fc_b")
scores = tf.nn.xw_plus_b(h_pool_flat, fc_W, fc_b)
predictions = tf.argmax(scores, 1)


# Build Test Data

In [8]:
df = pd.read_csv('/home/dima/Data/sentiment_manual_data_clean_test.csv')

In [16]:
X = []
Y = []
tokens_set = list(vocab)
pad_index = tokens_set.index('<pad>')


for i in range(len(df)):
    try:
        text = df['Text_clean'][i]
        sentiment = int(df['Sentiment'][i])
        
        text_tokens = tokenizer.tokenize(text)
        d = [tokens_set.index(t) for t in text_tokens if t in tokens_set]
        
        for i in range(len(d), 42, 1):
            d.append(pad_index)
            
        X.append(d)
        d_y = np.zeros(3)
        
        d_y[sentiment+1] = 1
        
        Y.append(d_y)
    except:
        pass

X = np.array(X)
Y = np.array(Y)

print X.shape
print Y.shape


(5823, 42)
(5823, 3)


# Predict

In [17]:
sess = tf.InteractiveSession()
saver = tf.train.Saver()
saver.restore(sess, "/home/dima/Models/sentiment_cnn.ckpt")


In [18]:
preds = predictions.eval(feed_dict={ input_x : X}) 

In [25]:
correct_preds = np.argmax(Y,1)

In [28]:
print classification_report(correct_preds, preds)

             precision    recall  f1-score   support

          0       0.80      0.81      0.80      2337
          1       0.68      0.71      0.70      2217
          2       0.63      0.57      0.60      1269

avg / total       0.72      0.72      0.72      5823

